In [4]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
import time

In [ ]:
service = Service(r'C:\\Users\\Roshan Koirala\\Desktop\\FUN with GPT\\NEPAL PREMIER LEAGUE\\NPL NEW TRY\\geckodriver-v0.35.0-win64 (1)\\geckodriver.exe')

driver = webdriver.Firefox(service=service)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Firefox()

try:
    url = "https://www.espncricinfo.com/series/nepal-premier-league-2024-25-1462594/match-schedule-fixtures-and-results"
    driver.get(url)
    time.sleep(5)
    
    npl_matches = []
    
    for _ in range(5):  
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        links = driver.find_elements(By.CSS_SELECTOR, "a[href*='nepal-premier-league'][href*='full-scorecard']")
        for link in links:
            href = link.get_attribute('href')
            if href and href not in npl_matches:
                npl_matches.append(href)
    
    for match in npl_matches:
        print(match)
    print(f"\nTotal NPL matches: {len(npl_matches)}")

except Exception as e:
    print(f"Error: {str(e)}")
finally:
    driver.quit()

In [5]:
def extract_team_names(driver):
    team_elements = driver.find_elements(
        By.CSS_SELECTOR, ".ds-text-title-xs.ds-font-bold.ds-capitalize"
    )

    team_names = [team.text.strip() for team in team_elements]

    if len(team_names) >= 2:
        return team_names[0], team_names[1]
    else:
        return None, None

In [6]:
import re

In [7]:
def scrape_match_details(driver, match_url):
    driver.get(match_url)
    wait = WebDriverWait(driver, 10)

    table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ds-table')))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    match_details = {}
    rows = soup.find_all('tr')
    
    team1, team2 = extract_team_names(driver)
    team1 = team1.split(' ')[:2]
    team2 = team2.split(' ')[:2]
    
    match_details['Team 1'] = ' '.join(team1)
    match_details['Team 2'] = ' '.join(team2) 
       
    for row in rows:
        cols = row.find_all('td')
        if len(cols) == 2:  # two columns (label and value)
            label = cols[0].text.strip()
            value = cols[1].text.strip()
            
            if 'Toss' in label:
                toss_winner_name = value.split(' ')[:2]
                match_details['Toss_Winner'] = ' '.join(toss_winner_name)  
                match_details['Toss_Decision'] = value.split(' ')[-2]
            # elif 'Series' in label:
            #     match_details['Series'] = value
            elif 'Season' in label:
                match_details['Season'] = value
            elif 'Player Of The Match' in label:
                match_details['Player Of The Match'] = value
            elif 'Match days' in label:
                match_date = value.split(' ')[:3]
                match_details['Match_Day'] = ' '.join(match_date)
            elif 'Umpires' in label:
                umpires = [a.text.strip() for a in cols[1].find_all('a')]
                match_details['Umpires'] = umpires
            # elif 'TV Umpire' in label:
            #     match_details['TV Umpire'] = value
            # elif 'Points' in label:
            #     match_details['Points'] = value
    match_info_p = soup.find('p', {'class': 'ds-text-tight-s ds-font-medium ds-truncate ds-text-typo'})
    if match_info_p:
        match_details['Match Info'] = match_info_p.text.strip()
        
    team_divs = soup.find_all('div', {'class': 'ds-text-tight-m ds-font-bold ds-text-typo'})
    if len(team_divs) >= 2:  # Ensure two teams are present
        match_details['Team 1'] = team_divs[0].text.strip()
        match_details['Team 2'] = team_divs[1].text.strip()
    target_element = soup.find('div', {'class': 'ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap'})
    if target_element:
        target_value = target_element.find('strong').text.strip()
        match_details['Target'] = target_value
    else:
        match_details['Target'] = 'Not Available'
    
    
    #1st teeam`s run and wic lost by 1st one team
    match_details['Target_Runs'] = int(target_value.split('/')[0]) + 1 if '/' in target_value else int(target_value) + 1
    match_details['Wic_lost_by_1st_team'] = target_value.split('/')[1] if '/' in target_value else 10 
    
    
    # Extract runs scored by the second team (use a different element or class if necessary)
    runs_scored_element = soup.find_all('div', {'class': 'ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap'})
    if len(runs_scored_element) > 1:  # Assuming the second div contains runs scored by second team
        runs_scored = runs_scored_element[1].find('strong').text.strip()
        match_details['Second_Team_Performance'] = runs_scored
        match_details['Run_Scored_By_2nd_Team'] = runs_scored.split('/')[0]
        match_details['Wic_Lost_by_2nd_Team'] = runs_scored.split('/')[1] if '/' in runs_scored else 10
    else:
        match_details['Runs_Scored_By_Second_Team'] = 'Not Available'
    
    # Extract the run rates for both teams
    run_rates = re.findall(r'\(RR:\s([\d.]+)\)', driver.page_source)
    if len(run_rates) >= 2:
        match_details['Run_Rate_Team_1'] = run_rates[0]
        match_details['Run_Rate_Team_2'] = run_rates[1]
    else:
        match_details['Run_Rate_Team_1'] = 'Not Available'
        match_details['Run_Rate_Team_2'] = 'Not Available'
    
    
    
    return match_details

In [8]:
service = Service(r'C:\\Users\\Roshan Koirala\\Desktop\\FUN with GPT\\NEPAL PREMIER LEAGUE\\NPL NEW TRY\\geckodriver-v0.35.0-win64 (1)\\geckodriver.exe')

driver = webdriver.Firefox(service=service)

In [9]:
import requests

In [10]:
match_details = []

In [11]:
def extract_powerplay_info(url):
    try:
        # Fetch the HTML content from the match URL
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the <span> with the Powerplay info
        powerplay_spans = soup.find_all('span', string=lambda text: text and 'Powerplay' in text)

        if len(powerplay_spans) >= 2:
            # Get the first Powerplay info (first team's Powerplay performance)
            powerplay_1st_team_text = powerplay_spans[0].get_text()
            start_index_1st = powerplay_1st_team_text.find("Mandatory")
            if start_index_1st != -1:
                powerplay_1st_team_performance = powerplay_1st_team_text[start_index_1st:].split("Mandatory - ")[1].strip()
                match_details['powerplayperformance_1st_team'] = powerplay_1st_team_performance
            else:
                match_details['powerplayperformance_1st_team'] = "Powerplay info not found"

            # Get the second Powerplay info (second team's Powerplay performance)
            powerplay_2nd_team_text = powerplay_spans[1].get_text()
            start_index_2nd = powerplay_2nd_team_text.find("Mandatory")
            if start_index_2nd != -1:
                powerplay_2nd_team_performance = powerplay_2nd_team_text[start_index_2nd:].split("Mandatory - ")[1].strip()
                match_details['powerplayperformance_2nd_team'] = powerplay_2nd_team_performance
            else:
                match_details['powerplayperformance_2nd_team'] = "Powerplay info not found"
        else:
            match_details['powerplayperformance_1st_team'] = "Powerplay info not found"
            match_details['powerplayperformance_2nd_team'] = "Powerplay info not found"
    
    except requests.exceptions.RequestException as e:
        match_details['error'] = f"Error fetching Powerplay info: {e}"
    
    return match_details

In [ ]:
# matches = []

# for url in npl_matches:
#     match_details = scrape_match_details(driver, url)

#     match_info = match_details.get('Match Info', '')

#     if match_info:
#         match_info_parts = match_info.split(' by ')
        
#         if len(match_info_parts) == 2:  # Ensure 'by' splitting gives two parts
#             winner_team = match_info_parts[0]
#             won_by_margin = match_info_parts[1].strip()
            
#             if winner_team.startswith('S'):
#                 match_details['Winner_Team'] = ' '.join(winner_team.split(' ')[:2])
#             else:
#                 match_details['Winner_Team'] = winner_team.split(' ')[0]
            
#             match_details['won_by_margin'] = won_by_margin
    
       
       
#        ## powerplay
            
        
#     matches.append(match_details)
#     print(match_details)


#THE USEFUL CODEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE



matches = []

for url in npl_matches:
    match_details = scrape_match_details(driver, url)

    match_info = match_details.get('Match Info', '')

    if match_info:
        match_info_parts = match_info.split(' by ')
        
        if len(match_info_parts) == 2:  # Ensure 'by' splitting gives two parts
            winner_team = match_info_parts[0]
            won_by_margin = match_info_parts[1].strip()
            
            if winner_team.startswith('S'):
                match_details['Winner_Team'] = ' '.join(winner_team.split(' ')[:2])
            else:
                match_details['Winner_Team'] = winner_team.split(' ')[0]
            
            match_details['won_by_margin'] = won_by_margin
    
    ## Extract Powerplay info
    try:
        # Assuming you have a function to get the HTML content and parse the Powerplay info
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the Powerplay info using the 'Powerplay' text
        powerplay_info = soup.find('span', string=lambda text: text and 'Powerplay' in text)

        if powerplay_info:
            match_details['Powerplay_Info'] = powerplay_info.get_text()
            
        else:
            match_details['Powerplay_Info'] = "Powerplay info not found"
    except requests.exceptions.RequestException as e:
        match_details['Powerplay_Info'] = f"Error fetching Powerplay info: {e}"

    # Append the match details to the list
    matches.append(match_details)

    # Print the match details including Powerplay info
    print(match_details)



In [13]:
#MAIN FUNCTION FOR POWERPLAYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY

# def extract_powerplay_info(url):
#     try:
#         # Fetch the HTML content from the match URL
#         response = requests.get(url)
#         soup = BeautifulSoup(response.content, 'html.parser')

#         # Find the <span> with the Powerplay info
#         powerplay_info = soup.find('span', string=lambda text: text and 'Powerplay' in text)

#         if powerplay_info:
#             return powerplay_info.get_text()
#         else:
#             return "Powerplay info not found"
#     except requests.exceptions.RequestException as e:
#         return f"Error fetching data for {url}: {e}"

# # Loop through each match URL and extract the Powerplay info
# all_powerplay_info = []

# for match_url in npl_matches:
#     info = extract_powerplay_info(match_url)
#     match_details['PowerPlay'] = info


In [ ]:
import csv

csv_file_name = "Nepal_Premier_League.csv"

if matches:
    headers = matches[0].keys()  

with open(csv_file_name, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()  
    writer.writerows(matches)  

print(f"Match details have been written to {csv_file_name}")

In [ ]:
import pandas as pd
df = pd.read_csv('Nepal_Premier_League.csv')
df.head(5)